<a href="https://colab.research.google.com/github/junweiluo/HandsonMLPractice/blob/master/Chapter14Ex05_ClassificationAndLocalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

In [0]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [0]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [0]:
from functools import partial
import tensorflow as tf
from tensorflow import keras

def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [5]:
n_classes = info.features["label"].num_classes
n_classes # to be put into model as one parameter.

5

In [0]:
# transfer learning
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False) # exclude top layer
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = keras.layers.Dense(n_classes, activation="softmax")(avg) # specify output classes and corresponding activation.
loc_output = keras.layers.Dense(4)(avg) # specify output for bounding boxes.
# above is different from fully convolutoinal network, which is represented by YOLO.

model = keras.models.Model(inputs=base_model.input,
                           outputs=[class_output, loc_output]) # two outputs. unlike YOLO which has only one output multiclass.

In [0]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001) # This is using one step transfer.  A better use is at Ex03 Transfer learning. 
model.compile(loss=["sparse_categorical_crossentropy", "mse"], # can use different loss for different output in the same model.
              loss_weights=[0.8, 0.2], # depends on what you care most about
              optimizer=optimizer, metrics=["accuracy"])

In [0]:
def add_random_bounding_boxes(images, labels):
    fake_bboxes = tf.random.uniform([tf.shape(images)[0], 4])
    return images, (labels, fake_bboxes)

fake_train_set = train_set.take(5).repeat(2).map(add_random_bounding_boxes)

In [9]:
model.fit(fake_train_set, steps_per_epoch=5, epochs=2)

Epoch 1/2
5/5 [==============================] - 100s 20s/step - loss: 1.3050 - dense_loss: 1.5577 - dense_1_loss: 0.2942 - dense_accuracy: 0.2688 - dense_1_accuracy: 0.2750
Epoch 2/2
5/5 [==============================] - 100s 20s/step - loss: 1.1258 - dense_loss: 1.3592 - dense_1_loss: 0.1920 - dense_accuracy: 0.5500 - dense_1_accuracy: 0.2562
